In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import time
from os import listdir
from os.path import isfile, join

In [7]:
PATH_TO_DATABASE = "./dataset/"

In [8]:
data_files = []

for f in listdir(PATH_TO_DATABASE):
    if f.endswith(".gz"):
        data_files.append(f)
    
data_files

['lenta.ru_159b9f4b-972b-48b1-8ec3-44fbd6be33c4_01.gz',
 'lenta.ru_4deb864d-3c46-45e6-85f4-a7ff7544a3fb_01.gz',
 'lenta.ru_6398c7e2-16da-40d2-8923-95f65aaaeb07_01.gz',
 'lenta.ru_80e74243-83da-4367-8ae3-fe38d333f283_01.gz',
 'lenta.ru_aa5a1ef9-6ca4-4dc7-890f-308d4d62db59_01.gz',
 'lenta.ru_b6838708-1aa9-496f-bf88-e277374f93a8_01.gz',
 'lenta.ru_b81aa623-ba55-43dc-b3c5-47ae2253ad27_01.gz',
 'lenta.ru_d1f7e910-b5f1-4719-b724-090093e143fe_01.gz']

In [9]:
from src.indexer import Indexer
from src.ts_idx.docreader import DocumentStreamReader

In [10]:
indexer = Indexer("./storage/", flush_threshold=8)

flush_threshold -  1048576.0


In [11]:
import time
start = time.time()

for data_file in data_files:
    documents_stream = DocumentStreamReader(PATH_TO_DATABASE + data_file)
    indexer.index(documents_stream)

end = time.time()
print end - start

10001 2354657 35397116.0071549416


In [12]:
indexer.tmp_files

['./storage/tmp/1459842679.txt',
 './storage/tmp/1459842681.txt',
 './storage/tmp/1459842683.txt',
 './storage/tmp/1459842685.txt',
 './storage/tmp/1459842687.txt',
 './storage/tmp/1459842689.txt',
 './storage/tmp/1459842691.txt',
 './storage/tmp/1459842693.txt']

In [13]:
indexer.merge()

input  ['./storage/tmp/1459842679.txt', './storage/tmp/1459842681.txt', './storage/tmp/1459842683.txt', './storage/tmp/1459842685.txt', './storage/tmp/1459842687.txt', './storage/tmp/1459842689.txt', './storage/tmp/1459842691.txt', './storage/tmp/1459842693.txt']
   merging  ./storage/tmp/1459842679.txt ./storage/tmp/1459842681.txt
   merging  ./storage/tmp/1459842683.txt ./storage/tmp/1459842685.txt
   merging  ./storage/tmp/1459842687.txt ./storage/tmp/1459842689.txt
   merging  ./storage/tmp/1459842691.txt ./storage/tmp/1459842693.txt
input  ['./storage/tmp/1459842792.txt', './storage/tmp/1459842794.txt', './storage/tmp/1459842797.txt', './storage/tmp/1459842799.txt']
   merging  ./storage/tmp/1459842792.txt ./storage/tmp/1459842794.txt
   merging  ./storage/tmp/1459842797.txt ./storage/tmp/1459842799.txt
input  ['./storage/tmp/1459842802.txt', './storage/tmp/1459842806.txt']
   merging  ./storage/tmp/1459842802.txt ./storage/tmp/1459842806.txt


'./storage/tmp/1459842810.txt'

In [14]:
from src.indexer import IndexEncoder

In [15]:
index_encoder = IndexEncoder("./storage/inverted_index.bin", "./storage/term_dictionary.bin")

In [16]:
index_encoder.encode('./storage/tmp/1459842810.txt')

In [25]:
from src.searcher import Searcher

In [29]:
searcher = Searcher("./storage/inverted_index.bin", "./storage/term_dictionary.bin")

In [33]:
query_meta = searcher.get_terms_meta([u"или", u"договор", u"aa+"])
query_meta

{-1713006339: {'offset': 230,
  'pos': 716748,
  'token': u'\u0434\u043e\u0433\u043e\u0432\u043e\u0440'},
 131558889: {'offset': 2706, 'pos': 988723, 'token': u'\u0438\u043b\u0438'},
 1189356009: {'offset': 1, 'pos': 578, 'token': u'aa+'}}

In [34]:
searcher.get_term_documents(**query_meta[131558889])

array([   5,    9,   18, ..., 9985, 9985, 9990])

In [37]:
from src.bool_parser import BoolQueryParser

In [38]:
query = "!True & False | !(False | True) & True"

In [39]:
BoolQueryParser.parse_query(query)

!(True)
!(True) & False
False | True
!(False | True)
!(False | True) & True
!(True) & False | !(False | True) & True
